In [2]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt

In [10]:
IMAGE_SIZE = 128
def getDataset(path):
    return tf.keras.preprocessing.image_dataset_from_directory(
        path,
        labels='inferred',
        image_size=(IMAGE_SIZE, IMAGE_SIZE),
        color_mode='grayscale',
    )

# Construct the Columbia datasets
def invert(x,y): return (-x, y)

data = getDataset('../data/created_dataset')
data_inv = data.map(invert)

Found 7200 files belonging to 24 classes.


In [19]:
X = np.empty((14400, 128, 128, 1))
y = np.empty(14400)

i = 0
for batch, lbls in data.take(-1):
    X[i*32:(i+1)*32] = batch
    y[i*32:(i+1)*32] = 0
    i += 1

for batch, lbls in data_inv.take(-1):
    X[i*32:(i+1)*32] = batch
    y[i*32:(i+1)*32] = 1
    i += 1

In [25]:
l = [
    layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1, input_shape=(128,128,1))
    layers.Flatten(input_shape=(128,128,1)),
    
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.BatchNormalization(),

    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.BatchNormalization(),

    layers.Dense(10, activation='relu'),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    
    layers.Dense(1, activation='sigmoid')
]

model = tf.keras.Sequential(l)
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [24]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-2),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.evaluate(X, y, steps=5)
model.fit(train, validation_data=test, epochs=10, callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

5/5 [==============================] - 1s 228ms/step - loss: 11.8253 - accuracy: 0.7021
Epoch 1/10
150/150 [==============================] - 20s 137ms/step - loss: -207.7280 - accuracy: 0.0454 - val_loss: -337.4821 - val_accuracy: 0.0521
Epoch 2/10
150/150 [==============================] - 18s 120ms/step - loss: -1674.8381 - accuracy: 0.0398 - val_loss: -3672.2175 - val_accuracy: 0.0425
Epoch 3/10
150/150 [==============================] - ETA: 0s - loss: -4782.7827 - accuracy: 0.0421

KeyboardInterrupt: 

In [ ]:
def plot_history(history, title_acc, title_loss, plot_name):
    acc = history.history['sparse_categorical_accuracy']
    val_acc = history.history['val_sparse_categorical_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.title(title_acc)
    plt.xlabel('Epoch')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.title(title_loss)
    plt.xlabel('Epoch')

    plt.savefig('./%s' % plot_name)
    
plot_history(history, 'Contrast augmentation', '', 'images/contrast_dropout.png')